In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
import plotly.express as px
import math

In [33]:
from PIL import Image
from stqdm import stqdm
class ClusterImage:
    def __init__(self, image_obj):
        self.image_obj = image_obj
        self.shape = np.shape(np.array(self.image_obj))
        self.image_array = np.reshape(np.array(self.image_obj),(self.shape[0] * self.shape[1], self.shape[2]) )
        self.clustered_image_array = np.reshape(np.array(self.image_obj),(self.shape[0] * self.shape[1], self.shape[2]) )
        self.cluster_image_obj = None

    def get_clustered_image(self, clusters = 10, clustering_type = "KMeans"):
        match clustering_type:
            case "KMeans":
                # st.write('kmeans')
                kmeans = KMeans(n_clusters=clusters, random_state=0, max_iter = 10, n_init=1).fit(self.image_array)
                for i, pixel in enumerate(tqdm(self.image_array)):
                    cluster = kmeans.labels_[i]
                    self.clustered_image_array[i] = [int(samp) for samp in kmeans.cluster_centers_[cluster] ]
        
        self.cluster_image_obj = Image.fromarray(np.reshape(self.clustered_image_array, self.shape))

    def graph_color_space(self, clustered_colors=False):
        if clustered_colors:   
            image_info = self.clustered_image_array
            TITLE = "Reduced Image Colorspace"
        else:
            image_info = self.image_array
            TITLE = "Original Image Colorspace"
        colors = np.unique(image_info, axis=0)
        color_names = []
        for col in (colors):
            color_names.append('Unknown Color')
        data = list(map(list, zip(colors[:,0], colors[:,1],colors[:,2],color_names)))
        new_df = pd.DataFrame(data,columns=['Red','Green','Blue', "Name"])
        colfig = px.scatter_3d(new_df, x='Red', y='Green', z='Blue',
                    color='Blue', title= TITLE)
        return colfig
        
    
    def dbscan_helper(self):
        from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
        db = DBSCAN(eps=0.12, min_samples=1).fit(self.image_array)
        labels = db.labels_
        cluster_values = {}
        for l in tqdm(labels, desc="Collecting clusters"):
            if l in cluster_values.keys():
                np.append(cluster_values[l], self.image_array[l])
                pass
            else:
                cluster_values[l] = [self.image_array[l]]

        cluster_values_avg = {}
        for l in tqdm(labels, desc="Averaging cluster members"):
            if l in cluster_values.keys():
                # print(cluster_values[l])
                cluster_values_avg[l] = np.mean(cluster_values[l], axis=0)

        for i, label in enumerate(tqdm(labels,desc="Generating new image...")):
            self.clustered_image_array[i] = [int(samp) for samp in cluster_values_avg[label] ]

In [34]:
immy = Image.open("rainb.jpg")
imageClustered = ClusterImage(immy)

# immy.save("saved.png")
# # print (imageClustered.image_array)

# imageClustered.get_clustered_image(10)

# # imageClustered.cluster_image_obj.save("savedCluster.png")
# image_info = imageClustered.image_array

In [35]:
imageClustered.dbscan_helper()

TypeError: stqdm.display() got an unexpected keyword argument 'check_delay'

In [5]:
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering

db = DBSCAN(eps=0.12, min_samples=1).fit(image_info)
labels = db.labels_

In [26]:
cluster_values = {}
for l in tqdm(labels, desc="Collecting clusters"):
    if l in cluster_values.keys():
        np.append(cluster_values[l], image_info[l])
        pass
    else:
        cluster_values[l] = [image_info[l]]

cluster_values_avg = {}
for l in tqdm(labels, desc="Averaging cluster members"):
    if l in cluster_values.keys():
        # print(cluster_values[l])
        cluster_values_avg[l] = np.mean(cluster_values[l], axis=0)


Averaging cluster members: 100%|██████████| 264600/264600 [00:06<00:00, 43101.48it/s]


In [9]:
from tqdm import tqdm

In [21]:
cluster_values[0]

[array([255,  92,  30], dtype=uint8)]

100%|██████████| 264600/264600 [00:06<00:00, 41016.55it/s]


11573

In [26]:
labset = set()

for lab in labels:
    labset.add(lab)

In [27]:
len(labset)

11573

In [19]:
len(image_info)

264600

In [10]:
labels

array([    0,     1,     1, ..., 11571, 11341, 11572], dtype=int64)

In [6]:
for i in range(0, len(pixels)):
    for j in range(0, len(pixels[0])):
        red = pixels[i][j][0]
        green = pixels[i][j][1]
        blue = pixels[i][j][2]

        print (j, ", ", i, ": " "R: ", red, " G: ", green, " B: ", blue)
        break
    break

# print(shape[0])


class Image:
    def __init__(self):
        self.clusterCount = 0
        self.U = 0  # total neighbors required
        self.W = 0  # width
        self.H = 0  # height
        self.pixelList = []  # All Pixels in Image
        self.pixelCluster = []  # All Clusters of Pixels

    def __init__(self, pixels):
        self.W = len(pixels[0])
        self.H = len(pixels)
        self.pixelList = []
        self.pixelCluster = []
        for i in range(0, len(pixels)):
            for j in range(0, len(pixels[0])):
                pair = [i, j]
                self.pixelList.append(Pixel(pixels[i][j], pair))
        self.clusterCount = 0

    def Distance(self, pointOne, pointTwo):  # Distance between points in 3D space
        tempVar = ((int(pointOne[0]) - int(pointTwo[0])) ** 2) + ((int(pointOne[1]) - int(pointTwo[1])) ** 2) + ((int(pointOne[2]) - int(pointTwo[2])) ** 2)
        d = math.sqrt(tempVar)
        return d

    def GetNeighbors(self, maxDist, curPixel):  # Sets Pixel's Neighbors below maxDist away in 3D space
        neighbors = []
        for i in range(0, len(self.pixelList)):
            if self.Distance(self.pixelList[i].returnRGB(), curPixel.returnRGB()) <= maxDist:
                if self.pixelList[i].returnCoord() != curPixel.returnCoord():
                    neighbors.append(self.pixelList[i])
        curPixel.setNeighbors(neighbors)

    def setCluster(self, U, curPixel, cluster):  # Begins new cluster (U is lowest number of pixels to be considered primary pixel)
        if (curPixel.returnNodeType() == 'n'):
            # curPixel.setCluster(cluster)
            ##Remove Pixel from Current image pixel list to shorten search time
            #print(curPixel.returnCoord())
            #print(curPixel.returnCoord()[0] * self.W + curPixel.returnCoord()[1])
            CurIndex = curPixel.returnCoord()[0] * self.W + curPixel.returnCoord()[1]

            self.pixelList.pop(CurIndex)

            # self.pixelList.remove(curPixel)
            if (curPixel.returnNeighborSize() > U):
                curPixel.setNodeType('p')

                tempNeighbors = curPixel.getNeighbors()
                for i in range(0, len(tempNeighbors)):
                    #self.GetNeighbors(4, tempNeighbors[i])
                    self.setCluster(U, tempNeighbors[i], cluster)
            else:
                curPixel.setNodeType('s')

    # def percentOfPixels Catagorized (if less than 90% make more random primary nodes)
    # def catagoryColor returns average color of that catagories pixel count (sum(r)/catagoryPixelCount, sum(g)/cPC, sum(b)/cPC)
    # when creating pixel, would it be faster to gen neighbors before or during catagorizing? prolly during (recursive)

# Distance(pointOne, pointTwo)

# We want user defined max distance (md stores furthest value from primary node to be converted to primary)
# and user defined sample nodes (if sn = 4, it needs to touch four pixels to be counted as a cluster node)
# class pixel (stores RGB value, stores which cluster, number of neighbors, if inserted as a primary node, secondary node, or not added)
# A way to handle outliers

# def findNeighbors()




curPixelTest = [255,112,34]
#totalNeighbor = findNeighborNum(pixels, curPixelTest, 0.5)
#print ("Count: ", totalNeighbor)

pixelList = []

for i in range(0, len(pixels)):
    for j in range(0, len(pixels[0])):
        pair = [i,j]
        pixelList.append(Pixel(pixels[i][j], pair))

#print(len(pixelList))
MyImg = Image(pixels)
testPixel = Pixel(pixels[0][0], [0,0])
MyImg.GetNeighbors(4, testPixel)
print(testPixel.returnNeighborSize())

testNeighbors = []
testNeighbors = testPixel.getNeighbors()
print("Test Pixel: ")
print(testPixel.returnRGB()[0], " ", testPixel.returnRGB()[1], " ", testPixel.returnRGB()[2])
print("_____________")
#for i in range(0, len(testNeighbors)): WORKS
    #print(testNeighbors[i].returnCoord()[0], " ", testNeighbors[i].returnCoord()[1], ": ", testNeighbors[i].returnRGB()[0], " ", testNeighbors[i].returnRGB()[1], " ", testNeighbors[i].returnRGB()[2])





U = 20
curCluster = 1
#print(len(MyImg.pixelList))
MyImg.setCluster(20, testPixel, 1)
print("Done")
#print(len(MyImg.pixelList))
#implement random sampling and if not within a cluster make a new cluster unless it has no neighbors (outlier)


IndexError: invalid index to scalar variable.